In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
# reading in results.csv
df = pd.read_csv('mini_batch_results.csv')

# drop useless columns
drop_cols = ['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward', 'RequesterAnnotation', 'Approve', 'Reject',
             'NumberOfSimilarHITs', 'AutoApprovalDelayInSeconds', 'AssignmentDurationInSeconds', 'LifetimeInSeconds']
drop_cols.extend(col for col in df.columns if 'Answer.opt' in col)
df = df.drop(drop_cols, axis=1)
df

,CreationTime,MaxAssignments,Expiration,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,...,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Answer.set_number,Answer.slider_values,Answer.slider_values2,Answer.srcc_flag,Answer.srcc_score
0,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,31LM9EDVOPTATOLPAPQU5RJVMSDJNO,A2312VA97WKLWC,Approved,Thu Mar 05 23:01:35 PST 2020,Thu Mar 05 23:29:44 PST 2020,Mon Mar 09 00:29:44 PDT 2020,Sat Mar 07 12:54:23 PST 2020,...,NaN,1689,100% (1/1),100% (1/1),100% (1/1),296,"9,23,34,13,16,20,16,17,23,26,22,23,27,27,31,28...","4,12,26,9,10,12,11,9,19,18,16,17,13,21,25,18,1...",pass,0.6668859288553503
1,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,31N2WW6R9VRNKI6J9GFUHFTXO06F3U,A1QMXXADPJEHRA,Rejected,Wed Mar 04 13:40:06 PST 2020,Wed Mar 04 13:52:00 PST 2020,Sat Mar 07 13:52:00 PST 2020,NaN,...,We have detected that you were not taking the ...,714,0% (0/1),0% (0/1),0% (0/1),224,"48,47,46,43,45,45,48,46,45,47,47,49,46,47,47,4...","43,42,41,37,42,34,33,43,41,44,45,45,38,44,44,3...",fail,0.051298917604257706
2,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,31QNSG6A5VUW6MXJ3ECEK4OR5SS87B,A2VKACLXTMOQWO,Approved,Mon Mar 02 23:10:08 PST 2020,Mon Mar 02 23:24:27 PST 2020,Thu Mar 05 23:24:27 PST 2020,Thu Mar 05 09:21:20 PST 2020,...,NaN,859,100% (1/1),100% (1/1),100% (1/1),7,"37,42,44,43,42,41,43,42,41,39,38,38,45,40,40,4...","32,33,37,34,36,34,33,36,32,34,35,35,40,37,36,3...",fail,0.2
3,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,326O153BMMZTWW8K7U10C6U929YEDK,A1TE0GFJUIVVCU,Rejected,Tue Mar 03 17:09:12 PST 2020,Tue Mar 03 17:23:58 PST 2020,Fri Mar 06 17:23:58 PST 2020,NaN,...,We have detected that you were not taking the ...,886,0% (0/1),0% (0/1),0% (0/1),127,"15,12,17,20,18,10,15,16,10,15,14,12,16,13,19,1...","10,8,12,13,11,6,9,9,5,4,5,8,8,8,9,7,7,7,9,7,7,...",fail,-0.7071067811865475
4,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,32EYX73OY4A4M2OBFVBPLDXG3WAURA,A1I7NC6QJH4PUF,Approved,Tue Mar 03 06:29:09 PST 2020,Tue Mar 03 07:02:51 PST 2020,Fri Mar 06 07:02:51 PST 2020,Tue Mar 03 09:21:28 PST 2020,...,NaN,2022,100% (1/1),100% (1/1),100% (1/1),57,"36,30,34,26,43,28,31,36,32,28,26,30,27,21,27,2...","24,20,24,18,33,21,17,28,21,21,18,23,21,16,16,2...",pass,0.9
5,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,32RIADZISW5H6JU9FEKCYJ4IRVG4SH,A1WRU2OBICVWO1,Approved,Fri Mar 06 08:01:48 PST 2020,Fri Mar 06 08:35:44 PST 2020,Mon Mar 09 09:35:44 PDT 2020,Sat Mar 07 12:54:23 PST 2020,...,NaN,2036,100% (1/1),100% (1/1),100% (1/1),305,"21,25,27,25,18,21,19,19,17,26,23,20,15,19,17,2...","19,21,20,20,14,21,17,15,14,22,16,13,14,17,12,1...",pass,0.9
6,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,32ZKVD547JOX71U34ETYYP2C8ZGB33,A29QSUM0DRB2QB,Approved,Tue Mar 03 22:17:35 PST 2020,Tue Mar 03 22:49:11 PST 2020,Fri Mar 06 22:49:11 PST 2020,Thu Mar 05 09:21:20 PST 2020,...,NaN,1896,100% (1/1),100% (1/1),100% (1/1),147,"36,36,35,28,37,39,39,36,40,36,33,39,39,36,40,3...","18,18,18,21,29,29,29,24,30,25,23,27,26,27,26,2...",fail,0.3
7,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,336YQZE83ZFW47L49M70R2BQYS75M4,AZFVZ2EJK1J0G,Approved,Tue Mar 03 09:32:57 PST 2020,Tue Mar 03 09:56:50 PST 2020,Fri Mar 06 09:56:50 PST 2020,Wed Mar 04 14:19:41 PST 2020,...,NaN,1433,100% (1/1),100% (1/1),100% (1/1),81,"18,17,25,37,25,25,26,16,19,23,34,28,30,27,35,3...","9,12,10,20,16,18,16,14,16,20,24,25,26,24,29,25...",pass,0.7
8,Mon Mar 02 22:08:57 PST 2020,129,Mon Mar 16 23:08:57 PDT 2020,33IZTU6J85284LIJZPMHVU1GKSXSX0,A1K0Z2TQZPOGZK,Rejected,Sat Mar 07 18:22:06 PST 2020,Sat Mar 07 18:35:29 PST 2020,Tue Mar 10 19:35:29 PDT 2020,NaN,...,We have detected that you were not taking the ...,803,0% (0/1),0% (0/1),0% (0/1),323,"39,38,39,41,39,38,38,36,35,34,38,42,38,40,39,3...","18,19,12,14,13,15,15,14,13,10,15,17,14,14,11,1...",fail,-0.3


In [ ]:
def slider_to_array(slider_vals):
    """ Convert slider_vals from output.csv to a Python array. Slider_vals 
    is expected to be a string of numbers separated by ','. Assumes the last
    char in slider_vals is ','."""
    try:
        vals = slider_vals[:-1].split(',')
        vals = [int(x) for x in vals]
        return(vals)
    except:
        return([])
    
def calc_slider_std(row, window_size=15, shift=5):
    """Calculate the smallest standard deviation out of all the windows
    in each slider. For each slider, splits the values into windows 
    of size window_size (where
    the start index is separated by 'shift' spaces) and calculates the 
    standard deviation for each window. Takes all standards deviations
    of both sliders and returns the min."""
    stds = []
    for slider_vals in [row['Answer.slider_values'], row['Answer.slider_values2']]:
        vals = slider_to_array(slider_vals)
        if len(vals) == 0:
            return([])
        i = 0
        while i < len(vals):
            end = min(len(vals), i+window_size)
            stds.append(np.std(vals[i:end]))
            i += shift
    return(min(stds))
    
def calc_slider_extreme(row, extreme):
    """ Calculates the min or max of all slider values. 
    If extreme == 'min', returns min value of all slider values from worker.
    If extreme == 'max', returns max value of all slider values from worker."""
    all_vals = []
    for slider_vals in [row['Answer.slider_values'], row['Answer.slider_values2']]:
        slider_vals = slider_to_array(slider_vals)
        all_vals.extend(slider_vals)
    if len(all_vals) == 0:
        return(-1)
    if extreme == 'min':
        return(min(all_vals))
    elif extreme == 'max':
        return(max(all_vals))
    else:
        raise ValueError("Expected 'min' or 'max' for parameter extreme, received %s." % extreme)
    
def predict(row, params):
    """Predicts whether a person's results should be Approved or Rejected based on the specified
    params."""
    if row['Answer.srcc_score'] == 'initial':
        return('reject')
    srcc_score = pd.to_numeric(row['Answer.srcc_score'])
    std = row[std_col]
    slider_min = row['slider_min']
    slider_max = row['slider_max']

    
    if (std < params['std_thresh']) or (srcc_score <= params['srcc_thresh']) or (slider_min < params['min_thresh']) or (slider_max > params['max_thresh']):
        return('Rejected')
    return('Approved')

These indices have low standard deviation: [1,38,18,29,37, 40, 41, 47, 62, 67, 71, 77, 80, 88, 90, 99, 108, 111, 119] 

These indices have somewhat low standard deviation: [38, 39, 75, 118] 

These indices have responses that are too low: [69, 78, 120] 


In [ ]:
# set these params
params = {
    'std_window': 15, # size of window used for calculating standard deviation
    'std_shift': 5, # how much to shift each window by (ex: window=15, shift=5 has windows [0, 14], [5, 19], [10, 24], ...)
    'std_thresh': 1, # lowest allowed standard deviation for a window
    'srcc_thresh': -0.5, # lowest allowed SRCC value
    'min_thresh': 4, # minimum allowed answer for any slider
    'max_thresh': 49, # maximum allowed answer for any slider
}

std_col = 'std_%i_%i' % (params['std_window'], params['std_shift'])
df[std_col] = df.apply(calc_slider_std, axis=1, window_size=params['std_window'], shift=params['std_shift']) # calc min std for all windows
df['slider_min'] = df.apply(calc_slider_extreme, axis=1, extreme='min') # calc min slider val
df['slider_max'] = df.apply(calc_slider_extreme, axis=1, extreme='max') # calc max slider val

df['predict'] = df.apply(predict, axis=1, params=params) # make predictions for approve/reject

In [ ]:
# predictions: column 'predict'
# actual: column 'AssignmentStatus'
df[['AssignmentStatus', 'predict', 'Answer.srcc_score', std_col, 'slider_min', 'slider_max']]

In [ ]:
# confusion matrix
actual = df['AssignmentStatus']
predicted = df['predict']
confusion_matrix(actual, predicted, labels=['Approved', 'Rejected'])

## Printing slider values for Rejected and Approved responses

In [ ]:
# rejected slider values
df[df['AssignmentStatus'] == 'Rejected'][['Answer.slider_values', 'Answer.slider_values2', 'Answer.srcc_score', std_col]].style.set_properties(subset=['Answer.slider_values', 'Answer.slider_values2'], **{'width': '300px'})

In [ ]:
# approved slider values
df[df['AssignmentStatus'] == 'Approved'][['Answer.slider_values', 'Answer.slider_values2', 'Answer.srcc_score', std_col]].style.set_properties(subset=['Answer.slider_values', 'Answer.slider_values2'], **{'width': '300px'})